In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. EDA
## 0) 필요한 라이브러리 임포트

In [ ]:
import numpy as np
import pandas as pd
import gc
import time
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)

## 1) 데이터셋 로드

In [ ]:
app_train = pd.read_csv('../input/home-credit-default-risk/application_train.csv')
app_test = pd.read_csv('../input/home-credit-default-risk/application_test.csv')
prev = pd.read_csv('../input/home-credit-default-risk/previous_application.csv')

In [ ]:
# 데이터 형태 살펴보기 : test데이터는 컬럼이 하나 적음
app_train.shape, app_test.shape, prev.shape

## 2) APP 데이터 살펴보기

### (1) Target 데이터 살펴보기, 0이면 제때 상환가능 / 1이면 상환이 어려운 것 의미함

In [ ]:
# 타겟값의 분포 : 타겟값의 분포가 92:8로 상당히 치우쳐져있음
print(app_train['TARGET'].value_counts())
print(24825/(282686+24825))

In [ ]:
#시각화로 확인
app_train['TARGET'].astype(int).plot.hist();

### (2) 결측치 살펴보기

In [ ]:
# 전체 결측치 개수 확인
mis_val=app_train.isnull().sum()
mis_val_percent=100*app_train.isnull().sum()/len(app_train)
mis_val_table=pd.concat([mis_val, mis_val_percent],axis=1)
mis_val_table_ren_columns=mis_val_table.rename(columns={0:'Missing Values',1:'% of Total Values'})
mis_val_table_ren_columns=mis_val_table_ren_columns[mis_val_table_ren_columns.iloc[:,1]!=0].sort_values('% of Total Values',ascending=False).round(1)
mis_val_table_ren_columns

### (3) 데이터 타입 살펴보기

In [ ]:
app_train.dtypes.value_counts()

In [ ]:
# 범주형데이터 (16개)의 경우, 범주가 몇개인지 살펴봄 : ORGANIZATION_TYPE 와 OCCUPATION_TYPE를 제외하고 카테고리수가 적은편
app_train.select_dtypes('object').nunique()

### (4) 컬럼별 특성 살펴보기

In [ ]:
# 주요 컬럼 시각화

#함수생성
def show(df, columns):
    cond_1 = (df['TARGET'] == 1)
    cond_0 = (df['TARGET'] == 0)
    
    for column in columns:
        fig, ax = plt.subplots(figsize=(12, 4), nrows=1, ncols=2, squeeze=False)
        sns.violinplot(x='TARGET', y=column, data=df, ax=ax[0][0])
        sns.distplot(df[cond_1][column], label='1', color='red', ax=ax[0][1])
        sns.distplot(df[cond_0][column], label='0', color='blue', ax=ax[0][1])

        
        
# Feature Importances 에서 중요하게 나왔던 칼럼만 가져옴
cols = ['AMT_INCOME_TOTAL','AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_ID_PUBLISH',
           'DAYS_REGISTRATION', 'DAYS_LAST_PHONE_CHANGE', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT', 'EXT_SOURCE_1', 
           'EXT_SOURCE_2', 'EXT_SOURCE_3', 'AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK', 
           'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR']

#시각화 확인
show(app_train, cols)

**주요 컬럼 확인결과**
- 'AMT_INCOME_TOTAL' : 0과 1사이에 분포 차이가 확연함 -> 다양한 변수로 가공가능
- 'DAYS_BIRTH' : 생일의 분포는 상당히 다양한편
- 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3' : 0과 1 사이의 분포 차이가 확연함, 

- 아래 피쳐들은 비율 등으로 가공할 수 있음
    - 'AMT_REQ_CREDIT_BUREAU_HOUR'
    - 'AMT_REQ_CREDIT_BUREAU_DAY'
    - 'AMT_REQ_CREDIT_BUREAU_WEEK' 
    - 'AMT_REQ_CREDIT_BUREAU_MON' 
    - 'AMT_REQ_CREDIT_BUREAU_QRT'
    - 'AMT_REQ_CREDIT_BUREAU_YEAR'

In [ ]:
# 위에서 확인한 주요 피쳐와 타겟의 상관계수 확인
print(app_train['DAYS_BIRTH' ].corr(app_train['TARGET']))
print(app_train['AMT_INCOME_TOTAL' ].corr(app_train['TARGET']))
print(app_train['EXT_SOURCE_1'].corr(app_train['TARGET']))
print(app_train['EXT_SOURCE_2'].corr(app_train['TARGET']))
print(app_train['EXT_SOURCE_3'].corr(app_train['TARGET']))


### (4)-1 DAYS-BIRTH 변수 살펴보기

In [ ]:
# 절대값을 취해서 다시 상관계수 확인 : 양의 상관에서 음의 상관으로 바뀜
app_train['DAYS_BIRTH']=abs(app_train['DAYS_BIRTH'])
app_train['DAYS_BIRTH'].corr(app_train['TARGET'])

In [ ]:
# 일수를 365로 나눠서 연령으로 변경하여 재확인 : target이 0인 그룹의 경우, 20-30대에서 높은 것 확인됨
sns.kdeplot(app_train.loc[app_train['TARGET']==0,'DAYS_BIRTH']/365,label='target==0')
sns.kdeplot(app_train.loc[app_train['TARGET']==1,'DAYS_BIRTH']/365,label='target==1')
plt.xlabel('Age(year)')
plt.ylabel('Density')

In [ ]:
# 나이대별 그룹을 나누기
np.linspace(20,70,num=11)

#나이 기준 새로운 데이터프레임 생성
age_only=app_train[['TARGET','DAYS_BIRTH']]
age_only['YEARS_BIRTH']=age_only['DAYS_BIRTH']/365

# 나눠둔 그룹 기준으로 피쳐 생성
age_only['YEARS_GROUP']=pd.cut(age_only['YEARS_BIRTH'],bins=np.linspace(20,70,num=11))
age_only.head(10)

In [ ]:
# 나이를 기준으로 Groupby (이때 target은 평균으로 계산)
age_groups  = age_only.groupby('YEARS_GROUP').mean()

# 데이터 시각화
plt.figure(figsize=(8,8))
plt.bar(age_groups.index.astype(str), 100*age_groups['TARGET']) #target에 100을 곱해서 시각화 용이성
plt.xticks(rotation=75);
plt.xlabel('Age Group(years)');
plt.ylabel('Failur to Reapy(%)')

연령을 그룹으로 나눠서서 시각화 해본 결과
- 나이가 젊을 수록 상환 비율이 떨어짐
- 20-25세, 25-30세 30-35세 : 10% 이상 상환 못 함
- 55-60세, 60-65세, 65-70세 : 5% 이하 상환 못 함

# 2. Feature engineering

ref : https://bkshin.tistory.com/entry/%EC%BA%90%EA%B8%80-5-Home-Credit-Default-Risk

In [ ]:
# 피쳐 가공 전 데이터 결합
apps = pd.concat([app_train, app_test])
print(apps.shape)

### 1) 상관 높은 피쳐들의 재가공
위에서 높은 상관성을 확인한 피쳐들을 사용하여 평균, 표준편차 등을 계산하여 새로운 피쳐로 생성

In [ ]:
apps['APPS_EXT_SOURCE_MEAN'] = apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
apps['APPS_EXT_SOURCE_STD'] = apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)
apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APPS_EXT_SOURCE_STD']].head(10)

In [ ]:
# Null 값 채우기
apps['APPS_EXT_SOURCE_STD'] = apps['APPS_EXT_SOURCE_STD'].fillna(apps['APPS_EXT_SOURCE_STD'].mean())

### 2) 도메인을 바탕으로 한 피쳐 생성

* CREDIT_INCOME_PERCENT: 수입대비 대출금액(허가)의 백분율
* ANNUITY_INCOME_PERCENT: 수입대비 월 대출지급액의 백분율
* CREDIT_TERM: 지급 기간(총 대출금액과 월지급액을 기준으로 계산, 개월수)
* DAYS_EMPLOYED_PERCENT: 연령대비 고용일수 비율
* APPS_CREDIT_GOODS_DIFF : 대출금액(허가) 대비 소비자 대출 상품액
* APPS_CNT_FAM_INCOME_RATIO : 가족 수 대비 가처분 소득 비율

In [ ]:
apps['CREDIT_INCOME_PERCENT'] = apps['AMT_CREDIT'] / apps['AMT_INCOME_TOTAL']
apps['ANNUITY_INCOME_PERCENT'] = apps['AMT_ANNUITY'] / apps['AMT_INCOME_TOTAL']
apps['CREDIT_TERM'] = apps['AMT_ANNUITY'] / apps['AMT_CREDIT']
apps['DAYS_EMPLOYED_PERCENT'] = apps['DAYS_EMPLOYED'] / apps['DAYS_BIRTH']
apps['APPS_CREDIT_GOODS_DIFF'] = apps['AMT_CREDIT'] - apps['AMT_GOODS_PRICE']
apps['APPS_CNT_FAM_INCOME_RATIO'] = apps['AMT_INCOME_TOTAL']/apps['CNT_FAM_MEMBERS']


In [ ]:
# 새로 만든 피쳐들을 시각화
plt.figure(figsize=(12,20))
for i, feature in enumerate(['CREDIT_INCOME_PERCENT','ANNUITY_INCOME_PERCENT','CREDIT_TERM','DAYS_EMPLOYED_PERCENT','APPS_CREDIT_GOODS_DIFF','APPS_CNT_FAM_INCOME_RATIO']):
    plt.subplot(6,1,i+1)
    sns.kdeplot(apps.loc[apps['TARGET']==0,feature],label='target==0')
    sns.kdeplot(apps.loc[apps['TARGET']==1,feature],label='target==1')
    
    plt.title('Distribution of %s by Target Value' % feature)
    plt.xlabel('%s' % feature);
    plt.ylabel('Density');
    
plt.tight_layout(h_pad=2.5)


# 시각화 결과 target이 0일때와 1일때의 분포차이가 크지 않음
# 유의미한 피쳐가 될 수 있을지는 미지수임

# 3. Modeling

### 1) 데이터 재가공

In [ ]:
# 범주형 컬럼을 선택해서 레이블 인코딩
object_columns = apps.dtypes[apps.dtypes=='object'].index.tolist()

for column in object_columns:
    apps[column] = pd.factorize(apps[column])[0]

In [ ]:
# Train Test 데이터로 다시 나누기
apps_train = apps[-apps['TARGET'].isnull()]
apps_test = apps[apps['TARGET'].isnull()]
apps.shape, apps_train.shape, apps_test.shape

In [ ]:
from sklearn.model_selection import train_test_split

# Target 데이터 나누기
ftr_app = apps_train.drop(['SK_ID_CURR', 'TARGET'], axis=1)
target_app = app_train['TARGET']

#Train Set과 Validation Set으로 나누기
train_x, valid_x, train_y, valid_y = train_test_split(ftr_app, target_app, test_size=0.3, random_state=2020)
train_x.shape, valid_x.shape

### 2) 모델생성, 학습

In [ ]:
# pip install ligthgbm
from lightgbm import LGBMClassifier

In [ ]:
   clf = LGBMClassifier(
       nthread=4,
       n_estimators=10000,
       learning_rate=0.02,
       num_leaves=32,
       colsample_bytree=0.9497036,
       subsample=0.8715623,
       max_depth=8,
       reg_alpha=0.04,
       reg_lambda=0.073,
       min_split_gain=0.0222415,
       min_child_weight=40,
       silent=-1,
       verbose=-1)

In [ ]:
clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric= 'auc', verbose= 100, 
        early_stopping_rounds= 100)

In [ ]:
preds = clf.predict_proba(apps_test.drop(['SK_ID_CURR', 'TARGET'], axis=1))[:, 1 ]
app_test['TARGET'] = preds
app_test[['SK_ID_CURR', 'TARGET']].to_csv('submission.csv', index=False)

In [ ]:
app_test[['SK_ID_CURR', 'TARGET']].head()

In [ ]:
# plot_importance 시각화
from lightgbm import plot_importance
plot_importance(clf, figsize=(16, 32))

중요도를 살펴보면 도메인지식 기반으로 생성한 피쳐들이 중요한 역할을 하고있음을 알 수 있음